In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [11]:
csv_wc = "2018FIFAWC.csv"
wc_df = pd.read_csv(csv_wc)
wc_df.head()


,Team,Group,Squad Number,Position,Player,Date Of Birth,Age,Caps,Goals,Club,Player Count
0,Egypt,A,1,GK,Essam El-Hadary (captain),1973/01/15,45,157,0,Al-Taawoun,1
1,Egypt,A,2,DF,Ali Gabr,1989/01/01,29,20,1,West Bromwich Albion,1
2,Egypt,A,3,DF,Ahmed Elmohamady,1987/09/09,30,77,2,Aston Villa,1
3,Egypt,A,4,MF,Omar Gaber,1992/01/30,26,23,0,Los Angeles FC,1
4,Egypt,A,5,MF,Sam Morsy,1991/09/10,26,4,0,Wigan Athletic,1


In [9]:
csv_players = "players.csv"
players_df = pd.read_csv(csv_players)
players_df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
